In [1]:
import gradio as gr
import cv2
import numpy as np
from cvzone.ClassificationModule import Classifier
import math
from reportlab.pdfgen import canvas
import tempfile
import os

# Initialize classifier for Hibiscus vs Guava (Stage 1)
classifier = Classifier("Model2/keras_model.h5", "Model2/labels.txt")

# Parameters for leaf detection
imgSize = 300
labels = ["Hibiscus leaf", "Guava leaf"]
confidence_threshold = 0.5
lower_green = np.array([25, 30, 20])
upper_green = np.array([85, 255, 255])

# Descriptions for Hibiscus or Guava classification
plant_info = {
    'Hibiscus': {
        'description': 'This is a hibiscus leaf. Suitable for further analysis or gardening.',
        'cure': 'N/A'
    },
    'Guava': {
        'description': 'This is a guava leaf. Suitable for further analysis or gardening.',
        'cure': 'N/A'
    }
}


def process_image_for_classification(image):
    # Convert the uploaded image to OpenCV format
    image = np.array(image)

    # Convert to HSV color space for better leaf isolation
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, lower_green, upper_green)

    # Apply morphological operations to remove small noise
    kernel = np.ones((5, 5), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)

    # Find contours of the leaf in the masked area
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if contours:
        # Assume the largest contour is the leaf
        largest_contour = max(contours, key=cv2.contourArea)
        x, y, w, h = cv2.boundingRect(largest_contour)

        # Crop and resize the leaf region
        imgCrop = image[y:y + h, x:x + w]

        # Resize to fit the model input size
        imgWhite = np.ones((imgSize, imgSize, 3), np.uint8) * 255
        aspectRatio = h / w
        if aspectRatio > 1:  # Tall image
            scale = imgSize / h
            newWidth = math.ceil(scale * w)
            imgResize = cv2.resize(imgCrop, (newWidth, imgSize))
            wGap = math.ceil((imgSize - newWidth) / 2)
            imgWhite[:, wGap:newWidth + wGap] = imgResize
        else:  # Wide image
            scale = imgSize / w
            newHeight = math.ceil(scale * h)
            imgResize = cv2.resize(imgCrop, (imgSize, newHeight))
            hGap = math.ceil((imgSize - newHeight) / 2)
            imgWhite[hGap:newHeight + hGap, :] = imgResize

        # Perform leaf classification using the classifier model (Stage 1)
        prediction, index = classifier.getPrediction(imgWhite, draw=False)
        confidence_value = prediction[index]

        if confidence_value >= confidence_threshold:
            leaf_type = labels[index]
        else:
            leaf_type = "Unknown"

        return leaf_type, confidence_value * 100
    return "No leaf detected", 0


def classify_image(image):
    # Process the image to classify the leaf type using the existing classifier (Stage 1)
    leaf_type, confidence = process_image_for_classification(image)

    # Description for the classified leaf type (Stage 1)
    if leaf_type == "Hibiscus leaf":
        description = plant_info['Hibiscus']['description']
    elif leaf_type == "Guava leaf":
        description = plant_info['Guava']['description']
    else:
        description = "Could not detect a leaf."

    # Generate PDF report and save to a temporary file
    with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as tmp_file:
        pdf_path = tmp_file.name
        pdf = canvas.Canvas(pdf_path)
        pdf.drawString(100, 750, "Leaf Classification Report")
        pdf.drawString(100, 730, f"Leaf Type: {leaf_type}")
        pdf.drawString(100, 710, f"Confidence: {confidence:.2f}%")
        pdf.drawString(100, 690, f"Description: {description}")
        pdf.save()

    return leaf_type, confidence, description, pdf_path


# Gradio interface for leaf detection and classification with PDF report
interface = gr.Interface(
    fn=classify_image,
    inputs=gr.Image(type="numpy", label="Upload Leaf Image"),
    outputs=[
        gr.Textbox(label="Leaf Type (Stage 1)"),
        gr.Textbox(label="Confidence (%)"),
        gr.Textbox(label="Description"),
        gr.File(label="Download PDF Report")
    ],
    title="LeafID: AI-Powered Leaf Identification System",
    description="Upload an image of a leaf to classify it as Hibiscus or Guava and download the report.",
    theme="default"
)

# Launch the interface
interface.launch()


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


1/1 [==============================] - 0s 144ms/step
